In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import plotly.express as px
import plotly.graph_objects as go
import os

In [9]:
CURR_PATH = os.path.abspath(os.path.dirname('file'))

In [10]:
# count_char=pd.read_csv('/Users/yuouwu/Dropbox (MIT)/essay/empirical/raw/cc-est2019-alldata.csv')
county=pd.read_csv(CURR_PATH+'/data/raw/cc-est2019-alldata.csv')

In [11]:
pd.set_option("display.max_rows", 300) 
county_char=county
county_char.columns=county_char.columns.str.lower()
county_char=county_char[['year','county','state','stname','ctyname','agegrp','tot_pop','tot_male','tot_female','ba_male','ba_female']]

age_avg=['27','32','37','42','47','52','57','62']
#county_char['agegrp']=np.where([county_char['agegrp'].inrange(6,13)],'a','b')
county_char=county_char[(county_char['agegrp'].isin(range(4, 14))) | (county_char['agegrp']==0)]
county_char['agegrp']=np.where(county_char['agegrp']==0,'total',county_char['agegrp'])
for i,j in zip(age_avg,range(4,14)):
    county_char['agegrp']=np.where(county_char['agegrp']==str(j),i,county_char['agegrp'])
#year 2017-2019
county_char=county_char[county_char['year'].between(10, 12)]
county_char['county'] =county_char['county'].astype(str).str.zfill(3)
county_char['state'] =county_char['state'].astype(str).str.zfill(2)
county_char['fips']=county_char['state']+county_char['county']
to_add=county_char[county_char['agegrp']=='total']
ind_col=["year",'ctyname','county','stname','state','fips']
county_char=county_char.pivot(index=(ind_col), columns="agegrp",values='tot_pop')
county_char=county_char[age_avg].merge(to_add,on=('year','fips'),how='right')
county_char=county_char.drop(columns=['agegrp'])
county_char['black']=county_char['ba_male']+county_char['ba_female']

for i in ind_col:
    first_column = county_char.pop(i)
    county_char.insert(0, i, first_column)
    
first_column = county_char.pop('tot_pop')
county_char.insert(6, 'tot_pop', first_column)

for i in county_char.columns[7:]:
    county_char[i]= county_char[i]/ county_char['tot_pop']
county_char=county_char.drop(columns=['year'])
county_char=county_char.groupby(['fips']).mean().reset_index()


In [12]:
df=pd.read_csv(CURR_PATH+'/data/raw/ACS_income_2019/ACSST5Y2019.S1901_data_with_overlays_2021-08-15T185601.csv')
lb_earnings=pd.read_csv(CURR_PATH+'/data/temp/income_emp.csv')

In [13]:
pd.set_option('display.max_columns', None)
income=df
income['fips']=df['GEO_ID'].str[-5:]
# income=income.loc[:,income.columns.str.endswith(('E','fips'))]
income= income.loc[:, income.iloc[0].str.contains('^E.*!!Households!!Total|id|^E.*!!Households!!Median', regex=True)]
# income.columns = income.iloc[0].str.extract(r'(Total!!).*$')
income=income.drop(columns=['GEO_ID'])
new_name=["acs_total","Less than 10,000","10,000 to 14,999","15,000 to 24,999","25,000 to 34,999","35,000 to 49,999","50,000 to 74,999","75,000 to 99,999","100,000 to 149,999","150,000 to 199,999","200,000 or more",'median','fips']
income.columns = new_name
income=income[1:]
for i in new_name:
    if i!='fips':
        income[i]=income[i].astype(float)
income['income_22']=income[["Less than 10,000","10,000 to 14,999","15,000 to 24,999"]].sum(axis=1)/100
income['income_35']=income[["25,000 to 34,999"]].sum(axis=1)/100
income['income_48']=income[["35,000 to 49,999"]].sum(axis=1)/100
income['income_65']=income[["50,000 to 74,999"]].sum(axis=1)/100
income['income_over65']=income[["75,000 to 99,999","100,000 to 149,999","150,000 to 199,999","200,000 or more"]].sum(axis=1)/100
income_list=['22','35','48','65','over65']
# for i in income_list:
#     old_col='income_'+i
#     new_col=old_col+'_num'
#     income[new_col]=income['acs_total']*income[old_col]
income=income.loc[:,income.columns.str.startswith(('income','fips','total','median'))]
all_char=income.merge(county_char,on='fips',how='inner',indicator=False)
# all_char.loc[all_char['acs_total']!=all_char['tot_pop']]

lb_earnings['countyfips']=lb_earnings['countyfips'].astype(str)
lb_earnings['fips'] = lb_earnings['countyfips'].str.zfill(5)
mpc_df=all_char.merge(lb_earnings[['fips','annual_income','year','week','income_region']],on='fips',how='left',indicator=False)
mpc_df['annual_income2']=mpc_df['annual_income']
mpc_df['impute_flag']=np.where(mpc_df['annual_income2'].isnull()==True,1,0)
mpc_df['annual_income2']=np.where(mpc_df['impute_flag']==1,mpc_df['median'],mpc_df['annual_income2'])
# mpc_df.loc[mpc_df['annual_income2'].isnull()==True]
mpc_df['annual_income2'].describe()


count    194410.000000
mean      47933.222381
std       10034.071566
min       18462.777584
25%       41516.000000
50%       46757.000000
75%       53333.000000
max      127610.000000
Name: annual_income2, dtype: float64

In [14]:
income_dict = {'22': 0.2223, '35':0.1235,'48':0.0635,'65':0.0047,'over65':-0.035}

agempc=0.0045
agesquarempc=-0.0223
femalempc=-0.0873
blackmpc=0.2073
blackagempc=-0.0042
blackfemalempc=0.0128
mpc_df
income_list=['22','35','48','65','over65']
mpc_df['income_bracket']='over65'
mpc_df['income_bracket']=np.where(mpc_df['annual_income2']<=22000,'22',mpc_df['income_bracket'])
mpc_df['income_bracket']=np.where(((mpc_df['annual_income2']>22000) & (mpc_df['annual_income2']<=35000)),'35',mpc_df['income_bracket'])
mpc_df['income_bracket']=np.where(((mpc_df['annual_income2']>35000) & (mpc_df['annual_income2']<=48000)),'48',mpc_df['income_bracket'])
mpc_df['income_bracket']=np.where(((mpc_df['annual_income2']>48000) & (mpc_df['annual_income2']<=65000)),'65',mpc_df['income_bracket'])
                           
mpc_df['mpc_income']=None
for i in income_list:
    mpc_df['mpc_income']=np.where(mpc_df['income_bracket']==i,income_dict[i],mpc_df['mpc_income'])
    
age_list=[27,32,37,42,47,52,57,62]
for i in age_list:
    age_var=str(i)
    mpc_df['mpc_age']=0
    mpc_df['mpc_age']=agempc*mpc_df[age_var]+agesquarempc*(mpc_df[age_var]**2)+blackagempc*mpc_df['black']*mpc_df[age_var]
mpc_df['mpc']=mpc_df['mpc_income']+mpc_df['mpc_age']+blackmpc*mpc_df['black']+femalempc*mpc_df['tot_female']+blackfemalempc*mpc_df['ba_female']

mpc_df.to_excel(CURR_PATH+'/data/temp/mpc.xlsx', index = False)

In [27]:
# gdp 

# get regional GDP to calculate PPP share
import datetime
gdp_df=pd.read_csv(CURR_PATH+'/data/raw/CAGDP1.csv',header=None)
gdp_df.columns = gdp_df.iloc[4]
gdp_df = gdp_df.iloc[5:]
gdp_df['fips']=pd.to_numeric(gdp_df['GeoFips'],errors='coerce')
gdp_df=gdp_df.dropna(subset=['fips'])
gdp_df['fips']=gdp_df['GeoFips'].str.zfill(5)
gdp_df=gdp_df[['2019','fips']]
gdp_df=gdp_df[~gdp_df['2019'].str.contains("NA")]
gdp_df["gdp_2019"]=pd.to_numeric(gdp_df['2019'])
# US_total_GDP_2019=gdp_df['2019'].sum()
# gdp_df['gdp_weight']=gdp_df['2019']/US_total_GDP_2019
# gdp_df=gdp_df[['fips','gdp_weight']]
# gdp_df['temp']=1
gdp_df=gdp_df[['gdp_2019','fips']]
gdp_df.to_excel(CURR_PATH+'/data/temp/gdp_fips.xlsx', index = False)

# # the other gpd file
# gdp_df=gdp_df.loc[gdp_df['LineCode']==1]
# gdp_df['fips']=gdp_df['GeoFIPS'].str[2:7]
# gdp_df['2019']=pd.to_numeric(gdp_df['2019'],errors='coerce')
# # gdp_df['fips']=pd.to_numeric(gdp_df['fips'])
# # gdp_df['US_total_GDP_2019']=gdp_df.loc[0, ['2019']]
# US_total_GDP_2019=gdp_df.loc[0, ['2019']]
# gdp_df['gdp_weight']=gdp_df['2019']/US_total_GDP_2019[0]

wei=pd.read_excel(CURR_PATH+'/data/raw/weekly-economic-index_data.xlsx',sheet_name="2008-current")
wei.columns = wei.iloc[3]
wei=wei.iloc[4:][['Date','WEI']]
wei['datetime']=pd.to_datetime(wei['Date'], format='%m/%d/%Y')
wei['week']=wei['datetime'].dt.week
wei['year']=wei['datetime'].dt.year
wei=wei[wei['year']>=2020][['WEI','week','year']]
wei = wei.rename(columns={"WEI": "gdp_wei"})
# wei['temp']=1

# gdp_merge=gdp_df.merge(wei,on='temp',how='inner')
# gdp_merge['gdp']=gdp_merge['WEI']*gdp_merge['gdp_weight']
# gdp_merge=gdp_merge[['fips','gdp_2019','week','year','gdp_wei']]


wei.to_excel(CURR_PATH+'/data/temp/gdp.xlsx', index = False)
gdp_df
wei

<ipython-input-27-e67000fc7520>:34: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  wei['week']=wei['datetime'].dt.week


3,gdp_wei,week,year
630,1.68,1,2020
631,1.21,2,2020
632,1.46,3,2020
633,1.6,4,2020
634,1.98,5,2020
...,...,...,...
712,8.81,30,2021
713,8.1,31,2021
714,8.47,32,2021
715,7.93,33,2021


In [ ]:
# estimate elasticity of income to small business revenue
small_biz=pd.read_csv(CURR_PATH+'/data/raw/EconomicTracker/data/Womply - County - Weekly.csv')

small_biz['date']=(small_biz[['year','month','day_endofweek']].astype(str).agg('/'.join, axis=1))
small_biz['datetime']=pd.to_datetime(small_biz['date'], format='%Y/%m/%d')
small_biz['week']=small_biz['datetime'].dt.week
small_biz['year']=small_biz['datetime'].dt.year
small_biz['fips'] = small_biz['countyfips'].astype(str).str.zfill(5)

# mpc_biz=small_biz[['fips','revenue_all','week','year']].merge(mpc_df[['fips','year','week','mpc']],on=(['fips','week','year']))
small_biz.to_excel(CURR_PATH+'/data/temp/small_biz.xlsx', index = False)

# county_char=county_char[age_avg].merge(to_add,on=('year','fips'),how='right')